# Update and install libraries

In [0]:
!pip install gdown
!pip install gast==0.2.2

# Download model parameters

In [0]:
!gdown https://drive.google.com/uc?id=1VeItsWR5Pn6udTHiokWmLqt6StcyT2k5
!gdown https://drive.google.com/uc?id=10d8qcSGxGS5xEHYDoWvoUw87g0_QoJoq
!gdown https://drive.google.com/uc?id=10W5F7XO5gkzpxGalo2qgD_Ffzn3uhNO-
!gdown https://drive.google.com/uc?id=10cQBEnrMXM2T3vmBW0GIcWgHC2V3tofe

# Upload images
If these codelines crush, just execute them again.

In [0]:
from google.colab import files
uploaded = files.upload()
ts_urls = []
for x in uploaded:
  ts_urls.append(x)

# Select tensorflow 2.x

In [0]:
%tensorflow_version 2.x

# Import libraries

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import *
from tensorflow.keras.layers import *

import os


# Create test dataset

In [0]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

# Reescalar imagenes.
def resize(inimg, height, width):
  inimg = tf.image.resize(inimg, [height, width])
  return inimg

#Normalizas al rango [-1, +1] la imagen
def normalize(inimg):
  inimg = (inimg / 127.5) - 1
  return inimg

# Aumentación de datos : Random Crop + Flip
def random_jitter(inimg):
  inimg = resize(inimg, 286, 286)
  
  stacked_image = tf.stack([inimg], axis=0)
  cropped_image = tf.image.random_crop(stacked_image, size=[1, IMG_HEIGHT, IMG_WIDTH, 3])
  
  inimg = cropped_image[0]
  
  if tf.random.uniform(()) > 0.5:
    inimg = tf.image.flip_left_right(inimg)
  return inimg

@tf.function
def load_image(filename, augment=True):
  inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(filename)), tf.float32)[..., :3]
  
  inimg = resize(inimg,IMG_HEIGHT, IMG_WIDTH)
  
  if augment:
    inimg = random_jitter(inimg)
  
  inimg = normalize(inimg)
  
  return inimg


def load_test_image(filename):
  return load_image(filename, False)

test_dataset = tf.data.Dataset.from_tensor_slices(ts_urls)
test_dataset = test_dataset.map(load_test_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(1)


# Create generator model

In [0]:


def downsample(filters, apply_batchnorm=True):
  
  result = Sequential()
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  # Capa convulocional
  result.add(Conv2D(filters, 
                    kernel_size = 4,
                    strides = 2,
                    padding = "same",
                    kernel_initializer= initializer,
                    use_bias=not apply_batchnorm))
  
  # Capa de BatchNorm.
  if apply_batchnorm:
    result.add(BatchNormalization())
  
  # Capa de activación.
  result.add(LeakyReLU())
  
  return result

downsample(64)

In [0]:
def upsample(filters, apply_dropout=False):
  
  result = Sequential()
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  # Capa convulocional
  result.add(Conv2DTranspose( filters, 
                              kernel_size = 4,
                              strides = 2,
                              padding = "same",
                              kernel_initializer= initializer,
                              use_bias=False))
  
  # Capa de BatchNorm.
  result.add(BatchNormalization())
  
  # Capa de Dropout
  if apply_dropout:
    result.add(Dropout(0.5))
  
  # Capa de activación.
  result.add(ReLU())
  
  return result

upsample(64)

In [0]:
def Generator():
  
  inputs = tf.keras.layers.Input(shape=[None, None, 3])
  
  down_stack = [
      downsample(64, apply_batchnorm=False), # (bs, 128, 128, 64)
      downsample(128),                       # (bs, 64,  64,  128)
      downsample(256),                       # (bs, 32,  32,  256)
      downsample(512),                       # (bs, 16,  16,  512)
      downsample(512),                       # (bs, 8,   8,   512)
      downsample(512),                       # (bs, 4,   4,   512)
      downsample(512),                       # (bs, 2,   2,   512)
      downsample(512)                        # (bs, 1,   1,   512)
  ]
  
  up_stack = [
      upsample(512, apply_dropout=True),     # (bs, 2,   2,   512)
      upsample(512, apply_dropout=True),     # (bs, 4,   4,   512)
      upsample(512, apply_dropout=True),     # (bs, 8,   8,   512)
      upsample(512),                         # (bs, 16,  16,  512)
      upsample(256),                         # (bs, 32,  32,  256)
      upsample(128),                         # (bs, 64,  64,  128)
      upsample(64),                          # (bs, 128, 128, 64)
  ]
  
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  last = Conv2DTranspose(filters = 3,
                        kernel_size = 4,
                        strides = 2,
                        padding = "same",
                        kernel_initializer= initializer,
                        activation = "tanh")
  
  x = inputs
  s = [] #skip conections
  
  concat = Concatenate()
  
  for down in down_stack:
    x = down(x)
    s.append(x)
    
  s = reversed(s[:-1])
    
  for up, sk in zip(up_stack, s):
    
    x = up(x)
    x = concat([x, sk])
  
  last = last(x)
  
  return Model(inputs = inputs, outputs = last)

generator = Generator()

# Load model parameters

In [0]:

generator_optimizer     = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)

checkpoint_prefix = os.path.join("", "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                generator = generator)

checkpoint.restore(tf.train.latest_checkpoint(""))

# Generate images

In [0]:
def generate_images(model, test_input,save_filename = False, display_imgs = True):

  prediction = model(test_input, training=True)
  
  plt.figure(figsize=(10,10))

  display_list = [test_input[0],  prediction[0]]
  title = ['Input Image',  'Predicted Image']
  
  if display_imgs:
    for i in range(2):
      plt.subplot(1, 2, i+1)
      plt.title(title[i])
      # getting the pixel values between [0, 1] to plot it.
      plt.imshow(display_list[i] * 0.5 + 0.5)
      plt.axis('off')
  plt.show()

# Results

In [0]:
for inp in test_dataset:
      generate_images(generator, inp, False, display_imgs=True)